In [1]:
!ls /kaggle/input/aksharantar/aksharantar_sampled/hin

ls: cannot access '/kaggle/input/aksharantar/aksharantar_sampled/hin': No such file or directory


In [2]:
!pip install lightning wandb 

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split
import wandb
from pytorch_lightning.loggers import WandbLogger
import random
import torch.nn.functional as F

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# common_path = "/kaggle/input/aksharantar/aksharantar_sampled/hin/"
common_path = "./aksharantar_sampled/hin/"
train_path = common_path +'hin_train.csv'
test_path = common_path +'hin_test.csv'
val_path = common_path +'hin_valid.csv'

In [6]:
train_data = pd.read_csv(train_path)
val_data = pd.read_csv(val_path)

In [7]:
train_data.iloc[:, 1].values

array(['बिन्द्या', 'किरणकांत', 'यज्ञोपवीत', ..., 'असहमतों', 'सुलगायीं',
       'अंचुतेंगु'], dtype=object)

In [8]:
train_input = train_data.iloc[:, 0].values
val_input = val_data.iloc[:, 0].values
train_output = train_data.iloc[:, 1].values
val_output = val_data.iloc[:, 1].values

In [9]:
def get_dataloaders(path):
    data = pd.read_csv(path)
    data_input = data.iloc[:, 0].values
    data_output = data.iloc[:, 1].values
    return data_input, data_output

In [10]:
eng_to_int = {}
hin_to_int = {}
eng_to_int['SOW'] = 0
eng_to_int['EOW'] = 1
hin_to_int['SOW'] = 0
hin_to_int['EOW'] = 1
count = 2
for i in range(97, 123):
    eng_to_int[chr(i)] = count 
    count += 1
eng_to_int['PAD'] = count
count = 2
for i in range(2304, 2432):
    hin_to_int[chr(i)] = count
    count += 1
hin_to_int['PAD'] = count
eng_max_len = 26
hin_max_len = 22
def convert_word_to_tensor(word, language):
    if language == 'eng':
        char_to_int = eng_to_int
        max_len  =eng_max_len
    else:
        char_to_int = hin_to_int
        max_len = hin_max_len
    ascii = [char_to_int['SOW']]
    ascii += [char_to_int[c] for c in word]
    ascii.append(1) # for end of word
    for x in range(max_len - len(ascii)):
        ascii.append(char_to_int['PAD'])
    ascii = torch.tensor(ascii, device=device)
    # one_hot = [torch.zeros(1, 1, 256) for _ in ascii]
    # for i, c in enumerate(ascii):
        # one_hot[i][0][0][c] = 1
    return ascii

In [11]:
chr(2309)

'अ'

In [12]:
convert_word_to_tensor('शस्त्रागार', 'hin')

tensor([  0,  56,  58,  79,  38,  79,  50,  64,  25,  64,  50,   1, 130, 130,
        130, 130, 130, 130, 130, 130, 130, 130])

In [13]:
convert_word_to_tensor('अ', 'hin')

tensor([  0,   7,   1, 130, 130, 130, 130, 130, 130, 130, 130, 130, 130, 130,
        130, 130, 130, 130, 130, 130, 130, 130])

# Create dataloaders

In [14]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input, output):
        self.input = input
        self.output = output
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        return convert_word_to_tensor(self.input[idx], 'eng'), convert_word_to_tensor(self.output[idx], 'hin')

In [15]:
train_loader = CustomDataset(train_input, train_output)
val_loader = CustomDataset(val_input, val_output)
train_loader = DataLoader(train_loader, batch_size=128, shuffle=True)
val_loader = DataLoader(train_loader, batch_size=128, shuffle=True)

# Encoder

In [16]:
class Encoder(pl.LightningModule):
    def __init__(self, input_size, hidden_size, cell_type, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell_type = cell_type
        self.dropout = dropout
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = self.find_cell_type(cell_type)(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        

    def find_cell_type(self, cell_type):
        if cell_type == "GRU":
            return nn.GRU
        elif cell_type == "LSTM":
            return nn.LSTM
        else:
            return nn.RNN

    def forward(self, input, hidden):
        embedded = self.embedding(input).squeeze()
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def initHidden(self, batch_size=1):
        return torch.zeros(1, batch_size,  self.hidden_size, device=device)

In [17]:
enc = Encoder(len(eng_to_int) + 1, 64, "GRU", 0.1)
hidden = enc.initHidden(128)
for x, y in train_loader:
    print(x)
    print(y)
    out, hidden = enc(x, hidden)
    print(out)
    print(out.shape)
    break

tensor([[ 0, 20,  9,  ..., 28, 28, 28],
        [ 0, 20,  4,  ..., 28, 28, 28],
        [ 0,  4,  9,  ..., 28, 28, 28],
        ...,
        [ 0, 14,  2,  ..., 28, 28, 28],
        [ 0,  2,  2,  ..., 28, 28, 28],
        [ 0, 15, 10,  ..., 28, 28, 28]])
tensor([[  0,  56,  65,  ..., 130, 130, 130],
        [  0,  58,  79,  ..., 130, 130, 130],
        [  0,  28,  64,  ..., 130, 130, 130],
        ...,
        [  0,  48,  74,  ..., 130, 130, 130],
        [  0,   8,  38,  ..., 130, 130, 130],
        [  0,  42,  65,  ..., 130, 130, 130]])
tensor([[[-0.0130,  0.2128, -0.4074,  ..., -0.4605, -0.1165,  0.2566],
         [-0.0231,  0.3020, -0.3322,  ..., -0.3874,  0.1130,  0.3540],
         [ 0.0954,  0.0444, -0.5018,  ..., -0.2224, -0.0156,  0.0919],
         ...,
         [ 0.1694, -0.2720, -0.1199,  ...,  0.3555, -0.3045, -0.1979],
         [ 0.1698, -0.2691, -0.1140,  ...,  0.3520, -0.3065, -0.1920],
         [ 0.1702, -0.2671, -0.1094,  ...,  0.3490, -0.3083, -0.1881]],

        [[-0.0

# Decoder

In [18]:
class Decoder(pl.LightningModule):
    def __init__(self, output_size, hidden_size, cell_type, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.cell_type = cell_type
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = self.find_cell_type(cell_type)(hidden_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=1)

    def find_cell_type(self, cell_type):
        if cell_type == "GRU":
            return nn.GRU
        elif cell_type == "LSTM":
            return nn.LSTM
        else:
            return nn.RNN

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.rnn(embedded, hidden)
        linear_output = self.linear(output)
        # print("decoder: ", output, output.squeeze(0))
        prob_output = self.softmax(linear_output)
        # output = torch.argmax(output, dim=1)
        return prob_output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Train the encoder-decoder model

In [19]:
class EncoderDecoder(pl.LightningModule):
    def __init__(self, input_size, output_size, hidden_size, cell_type, dropout):
        super().__init__()
        self.encoder = Encoder(input_size, hidden_size, cell_type, dropout)
        self.decoder = Decoder(output_size, hidden_size, cell_type, dropout)

    def forward(self, input, target):
        _, hidden = self.encoder(input, self.encoder.initHidden(input.shape[0]))

        # Initialize the decoder hidden state with the encoder hidden state
        decoder_hidden = hidden

        # Initialize the decoder input with the <SOS> token
        decoder_input = target[:, 0]

        # Initialize the output sequence
        output_sequence = torch.zeros(target.shape[0], target.shape[1], self.decoder.linear.out_features, device=device)

        # Feed the decoder input and hidden state through the decoder for each token in the output sequence
        for i in range(1, target.shape[1]):
            decoder_output, decoder_hidden = self.decoder(decoder_input.unsqueeze(1), decoder_hidden)
            output_sequence[:, i] = decoder_output.squeeze(1)
            decoder_input = decoder_output.argmax(dim=2).squeeze(1)
        output_sequence =  output_sequence.transpose(1, 2)
        return output_sequence

    def forward_teacher_force(self, input, target):
        # Encode the input sequence to get the final hidden state
        _, hidden = self.encoder(input, self.encoder.initHidden(input.shape[0]))

        # Initialize the decoder hidden state with the encoder hidden state
        decoder_hidden = hidden

        # Initialize the decoder input with the <SOS> token
        decoder_input = target[:, 0]

        # Initialize the output sequence
        output_sequence = torch.zeros(target.shape[0], target.shape[1], self.decoder.linear.out_features, device=device)

        # Feed the decoder input and hidden state through the decoder for each token in the output sequence
        for i in range(1, target.shape[1]):
            decoder_output, decoder_hidden = self.decoder(decoder_input.unsqueeze(1), decoder_hidden)
            output_sequence[:, i] = decoder_output.squeeze(1)
            decoder_input = target[:, i]
        output_sequence =  output_sequence.transpose(1, 2)
        return output_sequence

    def training_step(self, batch, batch_idx):
        input, target = batch
        if random.random()  < 1:
            output = self(input, target)
        else: 
            output = self.forward_teacher_force(input, target)
        loss = F.cross_entropy(output, target)
        self.log('train_loss', loss, on_epoch=True, on_step=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input, target = batch
        output = self(input, target[:, :-1])
        loss = F.cross_entropy(output.reshape(-1, output.shape[-1]), target[:, 1:].reshape(-1))
        self.log('val_loss', loss, on_epoch=True, on_step=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


In [21]:
model = EncoderDecoder(len(eng_to_int)+1, len(hin_to_int)+1, 64, "RNN", 0.1)

In [22]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sooraj/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 10.2 K
1 | decoder | Decoder | 25.3 K
------------------------------------
35.6 K    Trainable params
0         Non-trainable params
35.6 K    Total params
0.142     Total estimated model params size (MB)
2023-05-04 23:36:48.685181: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib
2023-05-04 23:36:48.685199: I tensorflow/stream_executor/cuda/cudart_stu

Training: 0it [00:00, ?it/s]

/home/sooraj/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
for x,y in train_loader:
    model(x, y)

In [ ]:
def evaluate():
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_hidden = encoder.initHidden(input_length)

    # encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    encoder_output, encoder_hidden = encoder(
        input_tensor, encoder_hidden)
        # encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[[0]]] * target_length, device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_tensor.size(1)):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            # print(decoder_output.shape, target_tensor[:, di].shape)
            loss += criterion(decoder_output.squeeze(), target_tensor[:, di].squeeze())
            decoder_input = target_tensor[:, di].unsqueeze(1)  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden[di])
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di].squeeze())
            if decoder_input.item() == 1:
                break


In [ ]:
int_to_hin = {}
for x in hin_to_int.keys():
    int_to_hin[hin_to_int[x]] = x

In [ ]:
word = "ghar"
tensor = convert_word_to_tensor(word, 'eng')
h0 = torch.zeros(1, 64, device=device)
print(tensor.shape, h0.shape)
enc_out, enc_hidden = enc(tensor, h0)
dec_in = torch.tensor([[0]], device=device)
dec_hidden = enc_hidden.view(1, 1, 64)
print(dec_in.shape, dec_hidden.shape)
for i in range(10):
    dec_out, dec_hidden = dec(dec_in, dec_hidden)
    topv, topi = dec_out.topk(1)
    index = torch.argmax(dec_out)
    dec_in = torch.tensor([[index.item()]], device=device)
    print(int_to_hin[index.item()], end="")

torch.Size([26, 1]) torch.Size([1, 64])
torch.Size([1, 1]) torch.Size([1, 1, 64])
५ॲॲ्ंिफ़ूूँ

In [ ]:
int_to_hin[13]

'ऋ'

# Attention model

In [ ]:
class Attention(pl.LightningModule):
    def __init_(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.attn = nn.Linear(self.hidden_dim * 2, 1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, hidden, encoder_outputs):
        attn_weights = self.attn(torch.cat((hidden[0], encoder_outputs[0]), 1))
        attn_weights = self.softmax(attn_weights)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        return attn_applied
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_dim, device=device)